In [2]:
import tensorflow as tf
import numpy as np
!ls aclImdb/test
!ls aclImdb/train


labeledBow.feat neg             pos             urls_neg.txt    urls_pos.txt
labeledBow.feat pos             unsupBow.feat   urls_pos.txt
neg             unsup           urls_neg.txt    urls_unsup.txt


In [ ]:
!cat aclImdb/train/pos/6248_7.txt

In [4]:
!rm -r aclImdb/train/unsup

In [6]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [8]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I am very disappointed with "K-911." The original "good" quality of "K-9" doesn\'t exist any more. This is more like a sitcom! Some of casts from original movie returned and got some of my memory back. The captain of Dooley now loves to hit him like a scene from old comedy show. That was crazy. What\'s the deal with the change of Police? It seems like they are now LAPD! Not San Diego PD. It is a completely different movie from "'
0
b"Giallo fans, seek out this rare film. It is well written, and full of all sorts of the usual low lifes that populate these films. I don't want to give anything away, so I wont even say anything about the plot. The whole movie creates a very bizarre atmosphere, and you don't know what to expect or who to suspect. Recommended! The only place I've seen to get this film in english is from European Trash Cinema, for $15."
1
b"Terry Gilliam's and David Peoples' teamed up to create one of the most intelligent and creative science fiction movies of the '90's. Pe

In [9]:
import string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import re

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

max_features = 20000
embedding_dim = 128
sequence_length = 500

#instantiate text vectorization layer

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

text_ds = raw_train_ds.map(lambda x, y: x)

vectorize_layer.adapt(text_ds)

Two options to vectorize the data

In [10]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label
#vectorize the data
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
text_ds = raw_test_ds.map(vectorize_text)

#Do async prefetching /buffering of the data for best performance on GPU
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = train_ds.cache().prefetch(buffer_size=10)
test_ds = text_ds.cache().prefetch(buffer_size=10)


'Build a Model' Build 1D convnet Starting with an Embedding Layer

In [11]:
from tensorflow.keras import layers

#A integer input for vocab indices
inputs = tf.keras.Input(shape=(None,), dtype="int64")

#next add a layer to map those vocab indices into a space of dimensionality
#'embedding_dim'

x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

#Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalAveragePooling1D()(x)

#add a vanilla hidden layer
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

#project onto a single unit output layer, and squash it with sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="prediction")(x)

model = tf.keras.Model(inputs, predictions)

#Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

epochs = 3

model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3


2022-05-06 09:58:06.156933: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - ETA: 0s - loss: 0.3830 - accuracy: 0.8127

2022-05-06 09:58:21.695085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 18s 26ms/step - loss: 0.3830 - accuracy: 0.8127 - val_loss: 0.1618 - val_accuracy: 0.9508
Epoch 2/3
625/625 [==============================] - 16s 26ms/step - loss: 0.0808 - accuracy: 0.9699 - val_loss: 0.2122 - val_accuracy: 0.9154
Epoch 3/3
625/625 [==============================] - 16s 26ms/step - loss: 0.0234 - accuracy: 0.9928 - val_loss: 0.0909 - val_accuracy: 0.9671


In [12]:
model.evaluate(test_ds)

782/782 [==============================] - 4s 5ms/step - loss: 0.4927 - accuracy: 0.8736


[0.4927290976047516, 0.8735600709915161]

If we want to obtain a model capable of processing raw strings, we can create a new model (using the weights we just trained)


In [ ]:
#A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
#turn strings into vocabulary indices 
indices = vectorize_layer(inputs)
#turn vocab inde
